In [1]:
import pandas as pd 
from functools import reduce

# LOT 1: Données des hopitaux

## FICHIER DONNEE HOSPITALIERE SUR LE COVID-19 PAR DEPARTEMENT

In [2]:
def set_record_type(value):
    if value == 0:
        return "Total"
    elif value == 1:
        return "Homme"
    elif value == 2:
        return "Femme"
    else:
        return None

In [3]:
hosp_data_df = pd.read_csv("data/donnees-hospitalieres-covid19-2020-03-25-16h10.csv", delimiter=';', header=0, encoding="utf-8", names=['departement','type', 'date', 'nombre_hospitalisation', 'nombre_reanimation', 'nombre_renvoi_domicile', 'nombre_deces'])

In [5]:
hosp_data_df.head(6)

,departement,type,date,nombre_hospitalisation,nombre_reanimation,nombre_renvoi_domicile,nombre_deces
0,01,0,2020-03-18,2,0,1,0
1,01,1,2020-03-18,1,0,1,0
2,01,2,2020-03-18,1,0,0,0
3,02,0,2020-03-18,41,10,18,11
4,02,1,2020-03-18,19,4,11,6
5,02,2,2020-03-18,22,6,7,5


## FICHIER DONNEE ETABLISSEMENT SUR LE COVID-19 PAR DEPARTEMENT

In [6]:
# Contient le nombre d'établissements par jour s'occupant de patient atteint du COVID 19 par Département
hosp_data_etab_df = pd.read_csv("data/donnees-hospitalieres-etablissements-covid19-2020-03-25-16h10.csv", delimiter=';', names=['departement', 'date', 'nombre_etablissement'], encoding="utf-8", header=0)

In [18]:
hosp_data_etab_df.head()

,departement,date,nombre_etablissement
0,01,2020-03-18,1
1,02,2020-03-18,4
2,03,2020-03-18,2
3,04,2020-03-18,2
4,05,2020-03-18,2


In [8]:
hosp_data_etab_df['departement'] = hosp_data_etab_df['departement'].astype(str)
hosp_data_etab_df['nombre_etablissement'] = hosp_data_etab_df['nombre_etablissement'].astype(int)

## FUSION DES DATAS DU LOT 1

In [9]:
hosp_df = pd.merge(hosp_data_df, hosp_data_etab_df, on=['departement', 'date'], how='left')
#hosp_full_df['nombre_etablissement'] = hosp_full_df['nombre_etablissement'].fillna(0).astype(int)

In [10]:
print(len(hosp_df))
hosp_df.head(5)

2124


,departement,type,date,nombre_hospitalisation,nombre_reanimation,nombre_renvoi_domicile,nombre_deces,nombre_etablissement
0,01,0,2020-03-18,2,0,1,0,1.0
1,01,1,2020-03-18,1,0,1,0,1.0
2,01,2,2020-03-18,1,0,0,0,1.0
3,02,0,2020-03-18,41,10,18,11,4.0
4,02,1,2020-03-18,19,4,11,6,4.0


In [11]:
tot_df = hosp_df[hosp_df['type'] == 0].rename(columns={"nombre_hospitalisation": "nombre_hospitalisation_total", "nombre_reanimation": "nombre_reanimation_total", "nombre_renvoi_domicile": "nombre_renvoi_domicile_total", "nombre_deces": "nombre_deces_total"})
h_df = hosp_df[hosp_df['type'] == 1].rename(columns={"nombre_hospitalisation": "nombre_hospitalisation_h", "nombre_reanimation": "nombre_reanimation_h", "nombre_renvoi_domicile": "nombre_renvoi_domicile_h", "nombre_deces": "nombre_deces_h"})
f_df = hosp_df[hosp_df['type'] == 2].rename(columns={"nombre_hospitalisation": "nombre_hospitalisation_f", "nombre_reanimation": "nombre_reanimation_f", "nombre_renvoi_domicile": "nombre_renvoi_domicile_f", "nombre_deces": "nombre_deces_f"})
dfs = [tot_df, h_df, f_df]

df_final = reduce(lambda left, right: pd.merge(left,right,on=['departement', 'date', 'nombre_etablissement'], how='left'), dfs)
if len(df_final) != len(hosp_df)/3:
    print("Reduce Error")
    
df_final.head()

,departement,type_x,date,nombre_hospitalisation_total,nombre_reanimation_total,nombre_renvoi_domicile_total,nombre_deces_total,nombre_etablissement,type_y,nombre_hospitalisation_h,nombre_reanimation_h,nombre_renvoi_domicile_h,nombre_deces_h,type,nombre_hospitalisation_f,nombre_reanimation_f,nombre_renvoi_domicile_f,nombre_deces_f
0,01,0,2020-03-18,2,0,1,0,1.0,1,1,0,1,0,2,1,0,0,0
1,02,0,2020-03-18,41,10,18,11,4.0,1,19,4,11,6,2,22,6,7,5
2,03,0,2020-03-18,4,0,1,0,2.0,1,1,0,0,0,2,3,0,1,0
3,04,0,2020-03-18,3,1,2,0,2.0,1,3,1,0,0,2,0,0,2,0
4,05,0,2020-03-18,8,1,9,0,2.0,1,1,0,7,0,2,7,1,2,0


In [12]:
df_finall = df_final.drop(['type_x', 'type_y', 'type'], axis=1)

Index(['departement', 'type_x', 'date', 'nombre_hospitalisation_total',
       'nombre_reanimation_total', 'nombre_renvoi_domicile_total',
       'nombre_deces_total', 'nombre_etablissement', 'type_y',
       'nombre_hospitalisation_h', 'nombre_reanimation_h',
       'nombre_renvoi_domicile_h', 'nombre_deces_h', 'type',
       'nombre_hospitalisation_f', 'nombre_reanimation_f',
       'nombre_renvoi_domicile_f', 'nombre_deces_f'],
      dtype='object')
Index(['departement', 'date', 'nombre_hospitalisation_total',
       'nombre_reanimation_total', 'nombre_renvoi_domicile_total',
       'nombre_deces_total', 'nombre_etablissement',
       'nombre_hospitalisation_h', 'nombre_reanimation_h',
       'nombre_renvoi_domicile_h', 'nombre_deces_h',
       'nombre_hospitalisation_f', 'nombre_reanimation_f',
       'nombre_renvoi_domicile_f', 'nombre_deces_f'],
      dtype='object')


# LOT 2: Données des urgences et SOS medecins

## FICHIER DONNEE HOPITAUX PAR DEPARTEMENT

In [13]:
sursaud_data1_df = pd.read_csv("data/sursaud-covid19-quotidien-2020-03-25-15h46-feuille1.csv", delimiter=',', header=0, encoding="utf-8", names=['departement','date', 'tranche_age', 'nombre_passage_urgence_covid', 'nombre_passage_urgence_total', 'nombre_hospitalisation_passage_urgence_covid', 'nombre_passage_urgence_covid_h', 'nombre_passage_urgence_covid_f', 'nombre_passage_urgence_total_h', 'nombre_passage_urgence_total_f', 'nombre_hostalisation_covid_h', 'nombre_hostalisation_covid_f', 'nombre_acte_medical_covid', 'nombre_acte_medical_total', 'nombre_acte_medical_covid_h', 'nombre_acte_medical_covid_f', 'nombre_acte_medical_total_h', 'nombre_acte_medical_total_f'])

In [14]:
sursaud_data1_df.head()

,departement,date,tranche_age,nombre_passage_urgence_covid,nombre_passage_urgence_total,nombre_hospitalisation_passage_urgence_covid,nombre_passage_urgence_covid_h,nombre_passage_urgence_covid_f,nombre_passage_urgence_total_h,nombre_passage_urgence_total_f,nombre_hostalisation_covid_h,nombre_hostalisation_covid_f,nombre_acte_medical_covid,nombre_acte_medical_total,nombre_acte_medical_covid_h,nombre_acte_medical_covid_f,nombre_acte_medical_total_h,nombre_acte_medical_total_f
0,01,2020-02-24,0,0.0,357.0,0.0,0.0,0.0,202.0,155.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,01,2020-02-25,0,0.0,310.0,0.0,0.0,0.0,177.0,133.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,01,2020-02-26,0,0.0,366.0,0.0,0.0,0.0,193.0,173.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,01,2020-02-27,0,0.0,321.0,0.0,0.0,0.0,178.0,143.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,01,2020-02-28,0,0.0,339.0,0.0,0.0,0.0,166.0,173.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
hosp_final_df = pd.merge(sursaud_data1_df, df_finall, on=['departement', 'date'], how='left')
hosp_final_df.head()
#hosp_final_df[hosp_final_df['date'] == "2020-03-24"]


,departement,date,tranche_age,nombre_passage_urgence_covid,nombre_passage_urgence_total,nombre_hospitalisation_passage_urgence_covid,nombre_passage_urgence_covid_h,nombre_passage_urgence_covid_f,nombre_passage_urgence_total_h,nombre_passage_urgence_total_f,...,nombre_deces_total,nombre_etablissement,nombre_hospitalisation_h,nombre_reanimation_h,nombre_renvoi_domicile_h,nombre_deces_h,nombre_hospitalisation_f,nombre_reanimation_f,nombre_renvoi_domicile_f,nombre_deces_f
0,01,2020-02-24,0,0.0,357.0,0.0,0.0,0.0,202.0,155.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01,2020-02-25,0,0.0,310.0,0.0,0.0,0.0,177.0,133.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01,2020-02-26,0,0.0,366.0,0.0,0.0,0.0,193.0,173.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01,2020-02-27,0,0.0,321.0,0.0,0.0,0.0,178.0,143.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01,2020-02-28,0,0.0,339.0,0.0,0.0,0.0,166.0,173.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## FICHIER DONNEE HOPITAUX PAR REGION (INCOHERENCES)

In [16]:
#sursaud_data2_df = pd.read_csv("data/sursaud-covid19-quotidien-2020-03-25-15h46-feuille2.csv", delimiter=',', header=0, encoding="utf-8", names=['region','date_de_passage', 'tranche_age', 'nombre_passage_urgence_covid', 'nombre_passage_urgence_total', 'nombre_hospitalisation_passage_urgence_covid', 'nombre_passage_urgence_covid_h', 'nombre_passage_urgence_covid_f', 'nombre_passage_urgence_total_h', 'nombre_passage_urgence_total_f', 'nombre_hostalisation_covid_h', 'nombre_hostalisation_covid_f', 'nombre_acte_medical_covid', 'nombre_acte_medical_total', 'nombre_acte_medical_covid_h', 'nombre_acte_medical_covid_f', 'nombre_acte_medical_total_h', 'nombre_acte_medical_total_f'])

In [17]:
#sursaud_data2_df.head()

,region,date_de_passage,tranche_age,nombre_passage_urgence_covid,nombre_passage_urgence_total,nombre_hospitalisation_passage_urgence_covid,nombre_passage_urgence_covid_h,nombre_passage_urgence_covid_f,nombre_passage_urgence_total_h,nombre_passage_urgence_total_f,nombre_hostalisation_covid_h,nombre_hostalisation_covid_f,nombre_acte_medical_covid,nombre_acte_medical_total,nombre_acte_medical_covid_h,nombre_acte_medical_covid_f,nombre_acte_medical_total_h,nombre_acte_medical_total_f
0,1,2020-02-24,0,0.0,300.0,0.0,0.0,0.0,148.0,152.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2020-02-24,A,0.0,75.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2020-02-24,B,0.0,102.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2020-02-24,C,0.0,68.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,2020-02-24,D,0.0,20.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1,2020-02-24,E,0.0,35.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1,2020-02-25,0,0.0,235.0,0.0,0.0,0.0,113.0,122.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
7,1,2020-02-25,A,0.0,84.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1,2020-02-25,B,0.0,77.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1,2020-02-25,C,0.0,36.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
